In [1]:
!nvidia-smi

Tue May 12 14:27:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:07:00.0  On |                  N/A |
| 35%   34C    P5    25W / 260W |   1295MiB / 11016MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
import tensorflow_datasets as tfds

In [3]:
BATCH_SIZE = 16

SHUFFEL_SIZE = 1024

In [4]:
cnn_dailymail = tfds.load(name="cnn_dailymail")

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load dataset info from /home/yannik/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset cnn_dailymail (/home/yannik/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /home/yannik/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


In [5]:
train_tfds = cnn_dailymail['train']
test_tfds = cnn_dailymail['test']
val_tfds = cnn_dailymail['validation']

In [6]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [7]:
def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text.numpy().decode('UTF-8')

def tokenize_text(text, needs_sum):
    text = normalize_text(text)
    if needs_sum:
        text = "summarize: " + text
        ids = tokenizer.encode(text, return_tensors="tf", max_length=512) 
    else:
        ids = tokenizer.encode(text, return_tensors="tf", max_length=150)
    return tf.squeeze(ids)



def map_func(features):

    article_ids = tf.py_function(tokenize_text, inp=[features["article"], True], Tout=tf.int32)
    highlights_ids = tf.py_function(tokenize_text, inp=[features["highlights"], False], Tout=tf.int32)

    return article_ids, highlights_ids


In [8]:
train_ds = train_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

val_ds = val_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

test_ds = test_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  16449536  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  18881280  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  25176064  
Total params: 60,506,880
Trainable params: 60,506,880
Non-trainable params: 0
_________________________________________________________________


In [11]:
#x = next(iter(train_ds))
#model.fit({'inputs': x[0], 'decoder_input_ids':x[1]}, epochs=3)

In [12]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # prediction_scores: (bs, 150, 32128)
        # decoder_past_key_value_states: (bs, 512, 512), (bs, 8, 150, 64)
        # z: (bs, 512, 512)
        predictions, _, _ = model(x, decoder_input_ids=y, training=True)
        loss = loss_object(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y, predictions)

In [13]:
@tf.function
def val_step(x, y):
    predictions, _, _ = model(x,  decoder_input_ids=y, training=False)
    v_loss = loss_object(y, predictions)

    val_loss(v_loss)
    val_accuracy(y, predictions)

In [14]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

In [85]:
rouge_score = RougeScore()

rouge_score("I want some ice cream, what are you doing","What I want is ice cream, what the hell")
rouge_score.result()

rouge1 = 55.56, 95% confidence [55.56, 55.56]
rouge2 = 37.50, 95% confidence [37.50, 37.50]
rougeLsum = 55.56, 95% confidence [55.56, 55.56]


{'rouge1': 55.55555555555556, 'rouge2': 37.5, 'rougeLsum': 55.55555555555556}

In [86]:
@tf.function
def test_step(x, y):
    predictions, _, _ = model(x, decoder_input_ids=y, training=False)
    t_loss = loss_object(y, predictions)
    test_loss(t_loss)
    test_accuracy(y, predictions)  
    return tf.math.argmax(predictions, 2)




def predict(x, y):
    predicted_ids = model.generate(tf.reshape(x,(1,-1)))
    print(predicted_ids)
    decoded_prediction = tokenizer.decode(tf.reshape(predicted_ids, -1))
    decoded_label = tokenizer.decode(y)
    
    print("------")
    print("Pred Sentence:", decoded_prediction)
    print("True Sentence:", decoded_label)
    print("------")

In [88]:
for i, (x, y) in enumerate(test_ds):
    predicted_ids = test_step(x, y)
    
    for i in range(predicted_ids.shape[0]):
        decoded_prediction = tokenizer.decode(predicted_ids[i])
        print(predicted_ids[i])
        decoded_label = tokenizer.decode(y[i])
        rouge_score(decoded_label, decoded_prediction)
        predict(x[i],y[i])
    if i > 2:
        break
        

template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
print(template.format(epoch + 1,
                    train_loss.result(),
                    train_accuracy.result() * 100,
                    test_loss.result(),
                    test_accuracy.result() * 100))

print(rouge_score.result())

tf.Tensor(
[    3  1329  1483     3 22451     3 14143   152 12217    10  1214   270
    12   608     8  1588   934     3     5    50  7636   764    45  1187
  4394    12 21341    29     8 12268    52     7     3     5  5260    63
 11934  2091 10246   112   166  2373    13   758    47  6139     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64)
tf.Tensor([[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

tf.Tensor([[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int32)
------
Pred Sentence: 
True Sentence: white house said today that jeremy bernard would depart after a state dinner next month, confirming rumors he'd soon exit the administration. first lady offered a glowing review of bernards tenure, saying she was 'lucky to have such a talented individual on my team' and a 'lifelong friend he is moving back to california to think about what’s next and spend some quality time with garbo, my rescue beagle white house has not yet named a replacement.
------
tf.Tensor(
[    3  2047  1896     3    51  4695  5600     6 12210     6    45 17907
    63     6   160    17     7     6   141     3     9     3    29 15242
   223  1406     3     5   160     3   122   102  9552   160    34    38
   323    12     3     9  8248  5495   900  2693    13     8 14732     3
     5    68   255    31    26   373   118  1101    11  1676   552     8
  1406  1553    16  2038     3     5   255   

KeyboardInterrupt: 

In [17]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    val_batches = iter(val_ds)
    
    for i, (x, y) in enumerate(train_ds):
        train_step(x, y)
        if i % 200 == 0:
            x_val, y_val = next(val_batches)
            val_step(x_val, y_val)
            template = 'Epoch {}, i: {}, train loss: {}, train acc: {}, val loss: {}, val acc: {}'
            print(template.format(epoch + 1,
                        i,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        val_loss.result(),
                        val_accuracy.result() * 100))

    
    for x, y in test_ds:
        test_step(x, y)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))
    
    print(rouge_score.result())



/home/yannik/Dokumente/text_summarization/venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1, i: 0, train loss: 13.879189491271973, train acc: 3.0833334922790527, val loss: 13.301065444946289, val acc: 0.4583333134651184
Epoch 1, i: 200, train loss: 4.01372766494751, train acc: 40.46165084838867, val loss: 7.6712117195129395, val acc: 26.625001907348633
Epoch 1, i: 400, train loss: 2.6640079021453857, train acc: 55.75373840332031, val loss: 5.26310396194458, val acc: 47.666664123535156
Epoch 1, i: 600, train loss: 1.95513117313385, train acc: 66.70770263671875, val loss: 3.986793041229248, val acc: 59.92708206176758
Epoch 1, i: 800, train loss: 1.533756136894226, train acc: 73.76570892333984, val loss: 3.202876567840576, val acc: 67.67500305175781
Epoch 1, i: 1000, train loss: 1.2589645385742188, train acc: 78.45030212402344, val loss: 2.6737985610961914, val acc: 72.94444274902344
Epoch 1, i: 1200, train loss: 1.0672893524169922, train acc: 81.73553466796875, val loss: 2.295513391494751, val acc: 76.7202377319336
Epoch 1, i: 1400, train loss: 0.926160454750061, train 

KeyboardInterrupt: 